In [18]:
%matplotlib inline

# 神经网络
可以使用`torch.nn`包构建神经网络。

`nn`依赖于`autograd`来定义模型并对其进行微分。

`nn.Module`包含层，以及返回`output`的方法`forward(input)`。

# 卷积网
一个简单的前馈网络，它获取输入，将其一层又一层地馈入，然后最终给出输出。

神经网络的典型训练过程如下：
1. 定义具有一些可学习参数（或权重）的神经网络。
2. 遍历输入数据集。
3. 通过网络处理输入。
4. 计算损失（输出正确的距离有多远）。
5. 将梯度传播回网络参数。
6. 通常使用简单的更新规则来更新网络的权重：`weight = weight - learning_rate * gradient`

# 定义网络

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


只需要定义`forward`函数，就可以使用`autograd`自动定义`backward`函数（计算梯度），可以在`forward`函数中使用任何张量操作

模型的可学习参数由`net.parameters()`返回

In [20]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


尝试一个`32*32`随机输入。

注意：该网络的预期输入大小（LeNet）为`32*32`。

要在MNIST数据集上使用此网络，将图像从数据集中调整为`32*32`

In [21]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.1174,  0.1452,  0.1127, -0.1481, -0.0143, -0.1306, -0.0326,  0.0416,
          0.0574, -0.0901]], grad_fn=<AddmmBackward>)


使用随机梯度将所有参数和反向传播的梯度缓冲区归零

In [22]:
net.zero_grad()
out.backward(torch.randn(1, 10))

注意

`torch.nn`仅支持小批量。整个`torch.nn`包仅支持作为微型样本而不是单个样本的输入。

例如，`nn.Conv2d`采用`nSamples x nChannels x Heigt x Width`的4D张量。

如果只有一个样本，只需使用`input.unsqueeze(0)`添加一个假批量尺寸。

回顾一下至此见到的所有类。
1. `torch.Tensor`一个多维数组。支持诸如`backward()`的自动微分操作。同样保持相对于张量的梯度。
2. `nn.Module`神经网络模块。封装参数的便捷方法，并带有将其移动到GPU，导出，加载等的帮助器。
3. `nn.Parameter`一种张量，即将其分配为`Module`的属性时，自动注册为参数。
4. `autograd.Function`实现自动微分操作的正向和反向定义。每个`Tensor`操作都会创建至少一个`Function`节点，该节点连接到创建`Tensor`的函数，并且编码其历史记录。

# 损失函数
损失函数采用一对（输出，目标）输入，并计算一个值，该值估计输出与目标之间的距离。

`nn`包下有几种不同的损失函数。

一个简单的损失是`nn.MSELoss`，计算输入和目标之间的均方误差。

In [23]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.8272, grad_fn=<MseLossBackward>)


现在，如果使用`.grad_fn`属性向后跟随`loss`，则看到一个计算图，如下所示：

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
    -> view -> linear -> relu -> linear -> relu -> linear
    -> MSELoss
    -> loss

当我们调用`loss.backward()`时，整个图将被微分。

损失，并且图中具有`requires_grad=True`的所有张量将随梯度累积其`.grad`张量

In [25]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

# 反向传播
要计算反向传播误差，只需要做`loss.backward()`。

注意在做之前需要清除现有的梯度，否则梯度将累积到现有的梯度中。

在计算`loss.backward()`，看一下向后前后`conv1`的偏差梯度

In [26]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0065, -0.0035,  0.0021, -0.0042,  0.0100, -0.0030])


# 更新权重
最简单的更新规则是随机梯度下降（SGD）：

`weight = weight - learning_rate * gradient`

In [27]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

但是，在使用神经网络时，希望使用各种不同的更新规则，例如SGD，Nesterov-SGD，Adam，RMSProp等。

为实现此目的，构建了一个包：`torch.optim`，可实现所有这些方法

In [28]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

注意使用`optimizer.zero_grad()`将梯度缓冲区手动设置为零。

这是因为反向传播部分中累积了梯度。